In [33]:
import pandas as pd

import numpy as np

import pymongo

from slugify import slugify

In [4]:
MONGODB_URL = "mongodb://35.237.232.73:27017/"

client = pymongo.MongoClient(MONGODB_URL)

In [5]:
DATABASE_NAME = "justica_estadual"

COLLECTION_NAME = "processos-tjsp"

In [6]:
processos = list(client[DATABASE_NAME][COLLECTION_NAME].find().limit(100))

print(f"Total de processos {len(processos)}")

Total de processos 100


In [195]:
from datetime import datetime


def t_d(datetime_str):
    datetime_object = datetime.strptime(datetime_str, '%Y%m%d%H%M%S')
    return datetime_object

def f_d(datetime_object):
    return datetime_object.strftime("%d/%m/%Y %H:%M:%S")

def m_d(datetimes):
    dt = [d for d in datetimes if not pd.isnull(d)]

    if len(dt) == 0:
        return None

    return min(dt)
        
def d_d(date1, date2):
    if not pd.isnull(date1) and not pd.isnull(date2):
        days = abs(date2-date1).days
        return days
    
    return None

In [147]:
df_sgt_movimentos = pd.read_csv("sgt_movimentos.csv", sep=";")
df_sgt_assuntos = pd.read_csv("sgt_assuntos.csv", sep=";")

In [148]:
class Movimento():
    def __init__(self, movimentos, df):
        self.movimentos = movimentos
        self.df = df
    
    def data_movimento(self, codigo):
        for movimento in self.movimentos:
            codigo_nacional = 0
            
            movimento_nacional = movimento.get("movimentoNacional")
            if movimento_nacional:
                codigo_nacional = movimento_nacional.get("codigoNacional")
            else:
                movimento_local = movimento.get("movimentoLocal")
                if movimento_local:
                    codigo_nacional = movimento_local.get("codigoPaiNacional")

            if codigo == codigo_nacional:
                return t_d(movimento["dataHora"])

        return None
    
    def nome_coluna(self, codigo):
        df_f = self.df[self.df["codigo"] == codigo]

        if len(df_f) != 1:
            raise Exception(f"Código {codigo} não encontrado na tabela de movimentos")
            
        return slugify(f"{codigo}_{df_f['descricao'].values[0]}", separator="_")

In [149]:
class Assunto():
    def __init__(self, assuntos, df):
        self.assuntos = assuntos
        self.df = df
        
    @property
    def principal(self):
        assunto = [ a for a in self.assuntos if a["principal"] == True ]
        
        if len(assunto) != 1:
            print("Assunto principal ausente ou maior que 1")
            return None
            
        return assunto[0]["codigoNacional"]
    
    def descricao(self, codigo):
        if codigo is None:
            return None
            
        df_f = self.df[self.df["codigo"] == codigo]

        if len(df_f) != 1:
            raise Exception(f"Código {codigo} não encontrado na tabela de assuntos")
            
        return df_f["descricao"].values[0]

In [191]:
def build_df(documentos):
    
    processos = []
    
    for documento in documentos:
        numero = documento["dadosBasicos"]["numero"]
        classe_processual = documento["dadosBasicos"]["classeProcessual"]
        orgao_julgador_codigo = documento["dadosBasicos"]["orgaoJulgador"]["codigoOrgao"]
        
        orgao_julgador_nome = documento["dadosBasicos"]["orgaoJulgador"]["nomeOrgao"]
        
        ass = Assunto(documento["dadosBasicos"]["assunto"], df_sgt_assuntos)
        
        assunto_codigo = ass.principal
        assunto_descricao = ass.descricao(assunto_codigo)
        
        mv = Movimento(documento["movimento"], df_sgt_movimentos)
     
        processos.append(
            (
                numero, 
                classe_processual,
                orgao_julgador_codigo,
                orgao_julgador_nome,
                assunto_codigo,
                assunto_descricao,
                
                mv.data_movimento(1),
                mv.data_movimento(3),
                
                mv.data_movimento(26),
                mv.data_movimento(51),
                mv.data_movimento(848),
                mv.data_movimento(970),
                
                mv.data_movimento(1013),
                mv.data_movimento(11009),
                mv.data_movimento(12164),
                mv.data_movimento(11383),
                mv.data_movimento(12265),
                mv.data_movimento(11010),    
            )
        )

    df = pd.DataFrame(processos, columns=["numero", "classe_processual", "orgao_julgador_codigo",
                                          "orgao_julgador_nome", "assunto_codigo", "assunto_descricao",
                                          mv.nome_coluna(1),
                                          mv.nome_coluna(3),
                                          mv.nome_coluna(26),
                                          mv.nome_coluna(51),
                                          mv.nome_coluna(848),
                                          mv.nome_coluna(970),
                                          mv.nome_coluna(1013),
                                          mv.nome_coluna(11009),
                                          mv.nome_coluna(12164),
                                          mv.nome_coluna(11383),
                                          mv.nome_coluna(12265),
                                          mv.nome_coluna(11010),
                                          ])
    
    return df

df = build_df(processos)
df.head()

Assunto principal ausente ou maior que 1
Assunto principal ausente ou maior que 1


,numero,classe_processual,orgao_julgador_codigo,orgao_julgador_nome,assunto_codigo,assunto_descricao,1_magistrado,3_decisao,26_distribuicao,51_conclusao,848_transito_em_julgado,970_audiencia,1013_determinacao,11009_despacho,12164_outras_decisoes,11383_ato_ordinatorio,12265_expedida_certificada,11010_mero_expediente
0,01055445220098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,2011-02-10 00:00:00,2010-01-04 11:05:22,2013-09-17 17:16:00,2011-02-10,NaT,NaT,2012-04-02,None,2014-06-09 16:55:33,None,NaT
1,01055895620098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,2014-07-25 15:44:56,2010-01-04 11:33:38,2012-02-14 00:00:00,NaT,NaT,NaT,2011-03-25,None,2019-09-13 15:00:43,None,NaT
2,01055939320098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,2013-01-16 00:00:00,2010-01-04 11:35:39,2013-09-16 14:07:00,NaT,NaT,NaT,2011-03-04,None,2018-10-29 15:29:42,None,NaT
3,01055956320098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,NaT,2010-01-04 11:36:35,2013-02-21 00:00:00,NaT,NaT,NaT,2011-02-15,None,2019-04-04 11:40:15,None,NaT
4,01056146920098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,2011-02-17 00:00:00,2010-01-04 11:48:41,2012-06-15 00:00:00,NaT,NaT,NaT,2012-06-15,None,NaT,None,2019-02-27 18:27:16


In [187]:
df = df.astype(
    {
        "assunto_codigo": "Int64", 
        "1_magistrado": "datetime64",
        "11010_mero_expediente": "datetime64",
        "12164_outras_decisoes": "datetime64",
        "12265_expedida_certificada": "datetime64",
    })
df.head()

,numero,classe_processual,orgao_julgador_codigo,orgao_julgador_nome,assunto_codigo,assunto_descricao,1_magistrado,3_decisao,26_distribuicao,51_conclusao,...,1013_determinacao,11009_despacho,12164_outras_decisoes,11383_ato_ordinatorio,12265_expedida_certificada,11010_mero_expediente,t_distribuicao_1_audiencia,t_distribuicao_envio_conclusao,t_total_processo,t_distribuicao_primeiro_impulso
0,01055445220098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952,IPTU/ Imposto Predial e Territorial Urbano,NaT,2011-02-10 00:00:00,2010-01-04 11:05:22,2013-09-17 17:16:00,...,NaT,2012-04-02,NaT,2014-06-09 16:55:33,NaT,NaT,NaN,1352.0,401.0,401.0
1,01055895620098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952,IPTU/ Imposto Predial e Territorial Urbano,NaT,2014-07-25 15:44:56,2010-01-04 11:33:38,2012-02-14 00:00:00,...,NaT,2011-03-25,NaT,2019-09-13 15:00:43,NaT,NaT,NaN,770.0,NaN,NaN
2,01055939320098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952,IPTU/ Imposto Predial e Territorial Urbano,NaT,2013-01-16 00:00:00,2010-01-04 11:35:39,2013-09-16 14:07:00,...,NaT,2011-03-04,NaT,2018-10-29 15:29:42,NaT,NaT,NaN,1351.0,NaN,NaN
3,01055956320098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952,IPTU/ Imposto Predial e Territorial Urbano,NaT,NaT,2010-01-04 11:36:35,2013-02-21 00:00:00,...,NaT,2011-02-15,NaT,2019-04-04 11:40:15,NaT,NaT,NaN,1143.0,NaN,NaN
4,01056146920098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952,IPTU/ Imposto Predial e Territorial Urbano,NaT,2011-02-17 00:00:00,2010-01-04 11:48:41,2012-06-15 00:00:00,...,NaT,2012-06-15,NaT,NaT,NaT,2019-02-27 18:27:16,NaN,892.0,NaN,NaN


In [196]:
def calc_indicadores(row):
    """
    Tempo entre a distribuição e a 1ª audiência:
    - 1ª mov: 26 - Distribuição
    - 2ª mov: 970 - Audiência
    """
    row["t_distribuicao_1_audiencia"] = d_d(row["26_distribuicao"], row["970_audiencia"])
        
    """
    Tempo médio entre a distribuição e o envio para concluso:
    - 1ª movimentação: 26 - Distribuição
    - 2ª movimentação: 51 - Conclusão
    """
    row["t_distribuicao_envio_conclusao"] = d_d(row["26_distribuicao"], row["51_conclusao"])
    
    """
    Tempo total processo 
    - 26 - Distribuição 
    - 848 - Trânsito em julgado
    """
    row["t_total_processo"] = d_d(row["26_distribuicao"], row["848_transito_em_julgado"])
    
    """
    Tempo Distribuição e Primeiro impulso do processo
    - 26 - Distribuição
    
    - 3 Decisão
    - 11009 Despacho
    - 1013 Determinação
    - 12164 Outras Decisões
    - 1 Magistrado
    - 11010 Mero Expediente
    - 11383 Ato Ordinatório
    - 12265 Expedida/certificada
    """
    row["t_distribuicao_primeiro_impulso"] = d_d(row["26_distribuicao"], row["848_transito_em_julgado"])
    row["t_distribuicao_primeiro_impulso"] = d_d(row["26_distribuicao"], 
                                                m_d([row["3_decisao"], 
                                                    row["11009_despacho"],
                                                    row["1013_determinacao"],
                                                    row["12164_outras_decisoes"],
                                                    row["1_magistrado"],
                                                    row["11010_mero_expediente"],
                                                    row["11010_mero_expediente"],
                                                    row["11010_mero_expediente"],
                                                    ]))
    """
    Tempo médio entre a distribuição e a primeira ação do magistrado
    - 26 - Distribuição
    
    - 11010 Mero Expediente
    - 3 Decisão
    - 11009 Despacho
    - 1 Magistrado
    - 1013 Determinação
    - 12164 Outras Decisões    
    """
    row["t_distribuicao_acao_magistrado"] = d_d(row["26_distribuicao"], 
                                                m_d([row["3_decisao"], 
                                                    row["11009_despacho"],
                                                    row["1013_determinacao"],
                                                    row["12164_outras_decisoes"],
                                                    row["1_magistrado"],
                                                    row["11010_mero_expediente"]
                                                    ]))


    return row


df = df.apply(calc_indicadores, axis=1)
df.head()

,numero,classe_processual,orgao_julgador_codigo,orgao_julgador_nome,assunto_codigo,assunto_descricao,1_magistrado,3_decisao,26_distribuicao,51_conclusao,...,11009_despacho,12164_outras_decisoes,11383_ato_ordinatorio,12265_expedida_certificada,11010_mero_expediente,t_distribuicao_1_audiencia,t_distribuicao_envio_conclusao,t_total_processo,t_distribuicao_primeiro_impulso,t_distribuicao_acao_magistrado
0,01055445220098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,2011-02-10 00:00:00,2010-01-04 11:05:22,2013-09-17 17:16:00,...,2012-04-02,None,2014-06-09 16:55:33,None,NaT,NaN,1352.0,401.0,401.0,401.0
1,01055895620098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,2014-07-25 15:44:56,2010-01-04 11:33:38,2012-02-14 00:00:00,...,2011-03-25,None,2019-09-13 15:00:43,None,NaT,NaN,770.0,NaN,NaN,444.0
2,01055939320098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,2013-01-16 00:00:00,2010-01-04 11:35:39,2013-09-16 14:07:00,...,2011-03-04,None,2018-10-29 15:29:42,None,NaT,NaN,1351.0,NaN,NaN,423.0
3,01055956320098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,NaT,2010-01-04 11:36:35,2013-02-21 00:00:00,...,2011-02-15,None,2019-04-04 11:40:15,None,NaT,NaN,1143.0,NaN,NaN,406.0
4,01056146920098260651,1116,10770,01 CUMULATIVA DE VALPARAISO,5952.0,IPTU/ Imposto Predial e Territorial Urbano,None,2011-02-17 00:00:00,2010-01-04 11:48:41,2012-06-15 00:00:00,...,2012-06-15,None,NaT,None,2019-02-27 18:27:16,NaN,892.0,NaN,NaN,408.0
